In [ ]:
# df2.to_csv('Capstone2.csv',index=False)

# Latar Belakang

Sebuah perusahaan yang bergerak di bidang data, ingin merekrut *data scientist*. Perusahaan ini melakukan pelatihan dan melakukan perekrutan ke orang-orang yang berhasil menyelesaikan pelatihan tersebut. Banyak orang mendaftar ke pelatihan tersebut, tapi tidak semua yang ikut pelatihan berniat untuk pindah pekerjaan.   

## Pernyataan Masalah
Perusahaan ingin mengetahui **detail nya kejahatan di kota BOSTON**. Informasi ini akan membantu pihak kepolisian untuk detail nya kejahatan di kota boston.

Sebagai seorang *data analyst*, kita akan mencoba menjawab pertanyaan berikut:

**Bagaimana kejahatan di dalam kota boston dalam hitungan tahunan , bulanan , district**

# Data
Untuk menjawab pertanyaan di atas, kita akan menganalisa data kriminal yang sudah dikumpulkan oleh kepolisian. Dataset dapat diakses [di sini](https://www.kaggle.com/datasets/AnalyzeBoston/crimes-in-boston). 

In [ ]:
df2[df2['UCR'] == 'Part Two']

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

from datetime import datetime

import plotly.express as px 
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import warnings

Dataset ini berisi informasi terkait kriminalisasi beserta tgl kriminal tsb . Ada 17 kolom di dalam dataset crim.csv, yaitu:  
- INCIDENT_NUMBER               --> ID untuk kasus tersebut
- Offense_CODE                  --> Kode pelanggaran pada kepolisian Boston
- Offense_Code_Group            --> Jenis pelanggaran
- Offense_Description           --> Detail kasus pelanggaran
- District                      --> Wilayah
- Reporting_Area                --> No Kantor kepolisian
- Shooting                      --> Status cara pembunuhan
- Occured on Date               --> Tgl kasus diinput ke dalam data
- Year                          --> Tahun
- Month                         --> Bulan
- Day of week                   --> Hari
- Hour                          --> Jam
- Street                        --> Alamat
- Lat                           --> Sumbu Y
- Long                          --> Sumbu X
- Location                      --> Gabungan Sumbu X-Y



## Read Database

In [ ]:
df = pd.read_csv('crime.csv',encoding_errors='ignore')
df

In [ ]:
df1 = pd.read_csv('offense_codes.csv',encoding_errors='ignore')
df1

## Data Understanding and Cleaning
Sebelum masuk ke dalam analisis, kita perlu mengenal dataset kita lebih jauh dalam tahapan *data understanding*. Dari proses ini, kita akan tahu anomali-anomali apa saja yang terdapat di dalam dataset kita dan perlu ditangani dalam tahapan *data cleaning*. Setiap penangan anomali yang dilakukan, akan disertai dengan justifikasi langkah yang diambil, baik secara *domain knowledge* maupun secara statistik.

Pertama, mari kita lihat informasi dari dataset.

In [ ]:
display(df.head(),df.tail())

## RENAME COLUMNS 

In [ ]:

df = df.rename(columns={"INCIDENT_NUMBER": "IncidentID", "OFFENSE_CODE":"OffenseCode", "OFFENSE_CODE_GROUP":"OffenseCG",
                    "OFFENSE_DESCRIPTION": "OffenseDesc", "DISTRICT": "District", "REPORTING_AREA":"ReportingArea",
                    "SHOOTING":"Shooting","OCCURRED_ON_DATE":"OccurredOn","YEAR":"Year","MONTH":"Month","DAY_OF_WEEK":"Day",
                    "HOUR":"Hour","UCR_PART":"UCR","STREET":"Street","Lat":"Latitude","Long":"Longitude"})

In [ ]:
df['Date'] = pd.to_datetime(df['OccurredOn']).dt.date
df['Time'] = pd.to_datetime(df['OccurredOn']).dt.time
del df['OccurredOn']

## MISSING VALUE

Dibawah ini dapat dilihat di bagian column District , Shooting , Street , Latitude , Longtitude terdapat missing value 

In [ ]:
df.isnull().sum()

Sebelum menangani nilai null lainnya, saya memeriksa apakah ada nilai null dalam bentuk spasi atau simbol apa pun dan menggantinya dengan NaN (nilai null)

In [ ]:
df = df.replace(r'^\s*$', np.nan, regex=True) 

Setelah melakukan pengecekan , terdapat kenaikan missing value pada column ReportingArea

In [ ]:
df.isnull().sum()

Di sini, kami mengamati bahwa kolom 'ReportingArea' yang awalnya menunjukkan 0 nilai nol sekarang menunjukkan 20250 nilai null karena kolom ini memiliki spasi sebagai pengganti NaN. Oleh karena itu, kita juga perlu mengelola nilai yang hilang pada kolom ini.

Karena District,Reporting Area, dan Street adalah kolom yang terkait dan penting untuk analisis, kami memutuskan untuk menghapus semua baris yang nilai di ketiga kolom ini adalah nol.



In [ ]:
df = df.dropna(how='all', subset=['District', 'ReportingArea', 'Street']) 

In [ ]:
df.isnull().sum()

Kami mengamati bahwa setelah penghapusan baris dengan subset null, jumlah nilai yang hilang di tiga kolom, 'District', 'Reporting Area' dan 'Street' menurun.

In [ ]:
df.isna().sum()

In [ ]:
for column in ['District', 'UCR' , 'ReportingArea']:
    df[column].fillna(df[column].mode()[0], inplace=True)

## Disini saya mengisi data Shooting yg awalnya terdapat missing value

In [ ]:
#SHOOTING MISSING VALUE
Shooting_ = df["Shooting"].fillna('N' , inplace = True )

In [ ]:
df.isna().sum()

Saya juga mengabaikan nilai yang hilang yang tersisa di kolom Street karena kami memiliki nilai untuk District yang sesuai dan akan membantu dalam analisis. Oleh karena itu, ini adalah akhir dari preprocessing data.

## Delete duplicate database 


In [ ]:
df['IncidentID'].nunique()

In [ ]:
duplicate = df[df.duplicated()]
duplicate

In [ ]:
df1[df1['CODE'] == 3125]

In [ ]:
df2.drop(df2[df2['UCR'] == 'Other'].index,inplace=True)

In [ ]:
df.drop_duplicates(subset= 'IncidentID' , inplace= True)

In [ ]:
df.duplicated().sum()

Untuk Database 1 yg duplicate sudah clear , karena sudah di drop

### delete duplicate data base 2


In [ ]:
df1

In [ ]:
df1['CODE'].nunique()

In [ ]:
df1.shape

In [ ]:
df1.drop_duplicates(subset= 'CODE' , inplace= True)

Untuk database 2 yg duplikat juga sudah clear

## RENAME COLUMN DATABASE 2


In [ ]:

df1 = df1.rename(columns={"CODE": "OffenseCode" , "NAME": "OffenseDesc"})

In [ ]:
df1

## Gabungan antara database 1 dan 2

In [ ]:
df2= df.merge(df1, on='OffenseCode' , how= 'left')
df2

In [ ]:
df2[df2['OffenseCG'] == 'Other']

In [ ]:
df2.drop(columns = 'OffenseDesc_y' ,inplace=True )

## Menghapus nilai Other di dalam column OffenseCG

In [ ]:
df2.drop(df2[df2['OffenseCG'] == 'Other'].index,inplace = True)

In [ ]:
df2

In [ ]:
df2['OffenseCG'].nunique()

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns

In [ ]:
df2.isnull().sum()

## DATA VISUALIZATION

In [ ]:
OCG = df2[['OffenseCG']].value_counts()
OCG

In [ ]:
#Create Dataframe
Data1 = pd.DataFrame(OCG)
Data1

In [ ]:
#RENAME
df2.rename(columns={'OffenseDesc_x':'OffenseDesc'} , inplace=True)

In [ ]:
df2['OffenseCG'].nunique()

In [ ]:
df2

In [ ]:
df2[df2['Hour'] == 0]

In [ ]:
Data = Data1.reset_index().rename(columns={0 : 'counts'})
Data


In [ ]:
df2.duplicated().sum()

In [ ]:
df2.info()

In [ ]:
## Change houur 0 - 24
df2.loc[df2["Hour"] == 0, "Hour"] = 24

In [ ]:
df2['Hour'].unique()

## CATPLOT DATA ALL IN OFFENSE

In [ ]:
#Total Number offense crime in boston
plt.figure(figsize=(50,30))
sns.set_style("whitegrid")

sns.set(font_scale = 3)
viz1 = sns.catplot(x= 'counts', y='OffenseCG', height=50, kind='bar', data=Data)
plt.title('Total Offenses for each Offense Group')
plt.xlabel('Number of Offenses')
plt.ylabel('Offense Category Group')
plt.show()

## Number Offense per Year

Di bawah ini dapat diliat Grafik kasus di Boston berdasarkan tahun

In [ ]:
year_crime = df2[['Year']].value_counts()
year_crime

In [ ]:
Data2 = pd.DataFrame(year_crime).reset_index().rename(columns={0 : 'TCBY'}) #Total Crimes By Year
Data2

In [ ]:
sns.set(font_scale=1)
fig = plt.gcf()
fig.set_size_inches(10, 5)
ax = sns.lineplot(x="Year", y="TCBY",
                   marker='o', ci=None, data=Data2)

                   

plt.title('Number of Offenses over the years')
plt.xlabel('Years')
plt.ylabel('Number of Offenses')



plt.show()

## Number of Offenses / Month

    Di bawah ini dapat di liat juga kasus di Boston berdasarkan bulan


In [ ]:

sns.set(font_scale = 1)
fig = plt.gcf()
fig.set_size_inches(12, 8)
sns.countplot(data=df2, x='Month')
plt.ylabel('Number of Offenses')
plt.show()#MonthPlot

## Number Offense / District

        Grafik detail untuk total kasus di Boston berdasarkan District

In [ ]:
df2[df2['District']=='B2']

In [ ]:
fig = plt.gcf()
fig.set_size_inches(15, 8)
sns.countplot(data=df2, x='District') #DistrictPlot
plt.ylabel('Number of Offenses')

plt.show()

## Total Offense per hour

    Total kejahatan kasus di Boston berdasarkan jam

In [ ]:
fig = plt.gcf()
fig.set_size_inches(15, 8)
sns.countplot(data=df2, x='Hour') #DistrictPlot
plt.ylabel('Number of Offenses')

plt.show()

## TOP 8 total offense crime in Boston

    Di bawah ini saya melakukan analisa untuk 8 kasus kejahatan tertinggi di Boston

In [ ]:
O_Desc = df2[['OffenseCG']].value_counts()
O_Desc

In [ ]:
Data3 = pd.DataFrame(O_Desc).reset_index().rename(columns={0 : 'count(OffenseDesc)'}).head(8) #OffenseDesc
Data3

In [ ]:
fig = plt.gcf()
fig.set_size_inches(15, 8)
sns.set_style("whitegrid")
sns.set(font_scale = 5)
viz1 = sns.barplot(x='OffenseCG', y='count(OffenseDesc)', data=Data3)
plt.xlabel("Type of Offenses", fontsize=15)
plt.ylabel("Number of Offense", fontsize=15)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.title("Top 8 Offense Code Groups", fontsize=15)
plt.show()

## Total offense setiap tahun berdasarkan tahun dan jenis kasus di Boston

    Di bawah ini saya melakukan analisa untuk setiap kasus berdasarkan tahun 

In [ ]:
data4 = df2.groupby(['OffenseCG','Year']).count()[['IncidentID']].reset_index()


data4

In [ ]:
data4 = data4.rename(columns={'IncidentID' : 'count(OffenseDesc)'})
data4

In [ ]:
df4 = pd.merge(Data3,data4, on=['OffenseCG'] )
del df4['count(OffenseDesc)_x']
df4 = df4.rename(columns={'count(OffenseDesc)_y': 'Number of Offenses'})
df4

In [ ]:
# Cek total tabel
df2[(df2['OffenseCG'] == 'Motor Vehicle Accident Response') & (df2['Year'] == 2015)]

In [ ]:
df4

Di bawah ini dapat diliat langsung secara jelas untuk kenaikan dan penurunan kasus tersebut di setiap tahun.

In [ ]:
fig = plt.gcf()
fig.set_size_inches(20, 25)
ax = sns.lineplot(x="Year", y="Number of Offenses",
                  hue="OffenseCG", marker='o', ci=None, data=df4)
ax.legend(bbox_to_anchor=(0,0))
plt.title('Number of Offenses over the years')
plt.xlabel('Years')
plt.ylabel('Number of Offenses')
plt.show()


## Total kasus 'MOTOR VEHICLE ACCIDENT RESPONSE OFFENSES' berdasarkan bulan

    di bawah ini saya melakukan analisa lebih dalam lagi untuk tipe kasus 'MOTOR VEHICLE ACCIDENT RESPONSE OFFENSES' berdasarkan bulan

In [ ]:
df2[df2['OffenseCG'] == 'Motor Vehicle Accident Response']

In [ ]:
data5 = df2[df2['OffenseCG'] == 'Motor Vehicle Accident Response'][['OffenseCG','Month','Year']]
data5



In [ ]:
data6 = data5.groupby(['Year' , 'Month']).count()
data6

In [ ]:
#Cek Data apakah sudah benar
data6['OffenseCG'].sum()

In [ ]:
data6 = data6.reset_index()

In [ ]:
data6

Di bawah ini dapat di lihat untuk kasus 'MOTOR VEHICLE ACCIDENT RESPONSE OFFENSES' , di tahun 2016 bulan 6 terdapat kasus tertinggi

In [ ]:
fig = plt.gcf()
fig.set_size_inches(70, 40)
sns.set(font_scale=1.4)
ax = sns.barplot(x="Month", y="OffenseCG", hue="Year", data=data6)
ax.legend(loc='best', fontsize=50)
plt.title('Number of Motor Vehicle Accident Response Offenses over the months', fontsize = 100)
plt.xlabel('Months', fontsize=100)
plt.xticks(fontsize=50)
plt.yticks(fontsize=50)
plt.ylabel('Number of Offenses', fontsize = 100)
plt.show()

## TOP 3 OFFENSE CASE IN BOSTON 

## 1. TYPE OF MOTOR VEHICLE ACCIDENT RESPONSE
        di bawah ini saya melakukan analisa lebih dalam lagi , karena kasus 'MOTOR VEHICLE ACCIDENT RESPONSE' adalah yang tertinggi.

In [ ]:
data7 = df2[df2['OffenseCG'] == 'Motor Vehicle Accident Response'][['OffenseDesc','Latitude','Longitude']]
data7

In [ ]:
df2.drop(df2[df2['Longitude'] == -1.000000].index,inplace=True)

In [ ]:
data8 = data7.value_counts().reset_index()
data8.rename(columns={0 : 'Number Of Offenses','OffenseDesc' : 'Type of Motor Vehicle Offenses Case'},inplace = True)
data8


Di bawah ini terdapat grafik dari kasus 'MOTOR VEHICLE ACCIDENT RESPONSE' group , dan di dalam kasus 'MOTOR VEHICLE ACCIDENT RESPONSE' terdapat M/V property damage yang menjadi jenis kasus tertinggi di Boston

In [ ]:
fig = plt.gcf()
fig.set_size_inches(90, 40)
ax = sns.barplot(x='Type of Motor Vehicle Offenses Case', y='Number Of Offenses', data=data8)
plt.xlabel ('Type of Motor Vehicle Accident Case', fontsize=80)
plt.ylabel ('Number of Offenses', fontsize=80)
plt.xticks(rotation=15, fontsize=30)
plt.yticks(fontsize=50)
plt.show()


In [ ]:
import folium

## FOLIUM

In [ ]:
from folium import plugins

DI bawah ini terdapat peta dari boston yang menjelaskan detail dan lokasi kejadi tersebut , dan dapat diliat secara jelas

In [ ]:
m_vehicle = folium.Map(location = [latitude, longitude], zoom_start = 12)

offenses = plugins.MarkerCluster().add_to(m_vehicle)

for lat, lng in zip(data8.Latitude, data8.Longitude):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='lightgray', icon='pin'),
    ).add_to(offenses)

m_vehicle

## 2. LARCENY

    Larceny menjadi kasus ke2 tertinggi dan mari kita melakukan analsa lebih dalam lagi di bawah ini

In [ ]:
df2[df2['OffenseCG'] == 'Larceny']


In [ ]:
## CEK DATA LARCENY
df2[(df2['OffenseCG'] =='Larceny') & (df2['Year'] == 2015) & (df2['Month']==6)]

In [ ]:
data12 =df2[df2['OffenseCG'] == 'Larceny'][['OffenseDesc','Latitude','Longitude']].value_counts()
data12 =pd.DataFrame(data12).reset_index().rename(columns={0 :'Number Of Offenses'})



In [ ]:
data12

di bawah ini dapat diliat jenis kasus 'Larceny Shoplifting' menjadi kasus tertinggi di dalam group Larceny

In [ ]:
fig = plt.gcf()
fig.set_size_inches(90, 60)
ax = sns.barplot(x='OffenseDesc', y='Number Of Offenses', data=data12)
plt.xlabel ('Larceny', fontsize=80)
plt.ylabel ('Number of Offenses', fontsize=80)
plt.xticks(rotation=15, fontsize=30)
plt.yticks(fontsize=50)
plt.show()

Di bawah ini terdapat peta detail dan lokasi untuk kasus Larceny yang terjadi di kota Boston

In [ ]:
Larceny = folium.Map(location = [latitude, longitude], zoom_start = 12)

offenses = plugins.MarkerCluster().add_to(Larceny)

for lat, lng in zip(data12.Latitude, data12.Longitude):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='lightgray', icon='pin'),
    ).add_to(offenses)

Larceny



## MEDICAL ASSISTANCE
    Medical assitance menjadi kasus ke 3 tertinggi di Boston

In [ ]:
data13 =df2[df2['OffenseCG'] == 'Medical Assistance'][['OffenseDesc','Latitude','Longitude','District']].value_counts()
data13 =pd.DataFrame(data13).reset_index().rename(columns={0 :'Number Of Offenses'})
data13


Di dalam kasus Medical Assistance case group terdapat 'Sick/Injured/Medical-Person' yang menjadi kasus tertinggi dalam case group ini

In [ ]:
fig = plt.gcf()
fig.set_size_inches(90, 60)
ax = sns.barplot(x='OffenseDesc', y='Number Of Offenses', data=data13)
plt.xlabel ('Medical Assistance', fontsize=80)
plt.ylabel ('Number of Offenses', fontsize=80)
plt.xticks(rotation=15, fontsize=30)
plt.yticks(fontsize=50)
plt.show()

Untuk lokasi detail dan kejadi dalam kasus medical assistance group ada di bawah ini

In [ ]:
Medical_Ass = folium.Map(location = [latitude, longitude], zoom_start = 12)

offenses = plugins.MarkerCluster().add_to(Medical_Ass)

for lat, lng in zip(data13.Latitude, data13.Longitude):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='red', icon='pin'),
    ).add_to(offenses)

Medical_Ass